In [1]:
import sys, os

# このノートブックが test/ にあると仮定
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

# GraphicalPR をモジュール検索パスに追加
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# パス追加後にインポート
from graph import Wave
from core import UncertainArray

# テスト実行
w = Wave()
print(w.belief)

None
